# Acquire (acquire.py)

## Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

### acquire & summarize

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
    - ***Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for `airconditioningtypeid`.***
    - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
    - Only include properties that include a latitude and longitude value.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import os
from env import host, user, password

from sklearn.model_selection import train_test_split

In [ ]:
###################### Acquire Zillow Data ######################

def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
def new_zillow_data():
    '''
    This function reads the zillow data from the Codeup db into a df,
    write it to a csv file, and returns the df.
    '''
    sql_query = '''
            SELECT *
            FROM properties_2017 as prop 
            INNER JOIN (
                    SELECT id, p.parcelid, logerror, transactiondate
                    FROM predictions_2017 AS p
                    INNER JOIN (
                            SELECT parcelid,  MAX(transactiondate) AS max_date
                            FROM predictions_2017 
                            GROUP BY (parcelid)
                    ) AS sub
                        ON p.parcelid = sub.parcelid
                    WHERE p.transactiondate = sub.max_date
            ) AS subq
                ON prop.id = subq.id;
                 '''
    df = pd.read_sql(sql_query, get_connection('zillow'))
    df.to_csv('zillow.csv')
    return df

def get_zillow_data(cached=False):
    '''
    This function reads in titanic data from Codeup database if cached == False 
    or if cached == True reads in zillow df from a csv file, returns df
    '''
    if cached or os.path.isfile('zillow_df.csv') == False:
        df = new_zillow_data()
    else:
        df = pd.read_csv('zillow_df.csv', index_col=0)
    return df

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [ ]:
df = get_zillow_data()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an attribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
def missing_rows_df(df):
    '''Takes in a dataframe of observations and attributes 
    and returns a dataframe where each row is an attribute name, 
    the first column is the number of rows with missing values 
    for that attribute, and the second column is percent of total 
    rows that have missing values for that attribute'''
    d = {'num_rows_missing': df.isna().sum(), 
         'pct_rows_missing': df.isna().sum()/len(df)}
    new_df = pd.DataFrame(data=d)
    return new_df

In [ ]:
df_mr = missing_rows_df(df)
df_mr.head()

In [ ]:
cols_no_missing = df_mr[df_mr.num_rows_missing == 0].T.columns.to_list()
cols_no_missing

In [ ]:
cols_more_than_50pct_missing = df_mr[df_mr.pct_rows_missing > .5].T.columns.to_list()
cols_more_than_50pct_missing

takeaways: 

4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
def missing_cols_df(df):
    '''takes in a dataframe and returns a dataframe 
    with 3 columns: the number of columns missing, 
    percent of columns missing, and number of rows 
    with n columns missing'''
    d = {'num_cols_missing': df.isna().sum(axis=1), 
         'pct_cols_missing': df.isna().sum(axis=1)/len(df.columns),
         'num_rows': 'fill'}
    new_df = pd.DataFrame(data=d)
    return new_df

In [ ]:
missing_cols_df(df)

In [ ]:
df.isna().sum(axis=1).value_counts().index

### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [ ]:
prop = pd.Series([260, 261, 263, 273, 275, 279, 276])
df_su = df[(df.propertylandusetypeid.isin(prop))]
df_su.shape

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

    - The input:
        - A dataframe
        - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
        - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
    - The output:
        - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
    - hint:
        - Look up the dropna documentation.
        - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
        - Make use of inplace, i.e. inplace=True/False.

In [ ]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [ ]:
df1 = handle_missing_values(df_su, .85, .75)

In [ ]:
df1.head()

3. Decide how to handle the remaining missing values:

    - Fill with constant value.
    - Impute with mean, median, mode.
    - Drop row/column

In [ ]:
df2 = df1.dropna()
df2.isna().sum()

## Mall Customers

1. Acquire data from mall_customers.customers in mysql database.

In [ ]:
###################### Acquire Zillow Data ######################

def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
def new_mall_data():
    '''
    This function reads the mall data from the Codeup db into a df,
    write it to a csv file, and returns the df.
    '''
    sql_query = '''SELECT * FROM customers
                 '''
    df = pd.read_sql(sql_query, get_connection('mall_customers'))
    df.to_csv('mall_customers.csv')
    return df

def get_mall_data(cached=False):
    '''
    This function reads in mall data from Codeup database if cached == False 
    or if cached == True reads in mall df from a csv file, returns df
    '''
    if cached or os.path.isfile('mall_df.csv') == False:
        df = new_mall_data()
    else:
        df = pd.read_csv('mall_df.csv', index_col=0)
    return df

In [ ]:
mall = get_mall_data()
mall.head()

2. Summarize data (include distributions and descriptive statistics).

In [ ]:
mall.info()

In [ ]:
mall.drop(columns='customer_id').describe().T

In [ ]:
mall.drop(columns='customer_id').hist()
plt.tight_layout()

3. Detect outliers using IQR.

In [ ]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [ ]:
get_upper_outliers(mall.age, 1.5).sum()

In [ ]:
get_upper_outliers(mall.spending_score, 1.5).sum()

In [ ]:
get_upper_outliers(mall.annual_income, 1.5).sum()

In [ ]:
get_upper_outliers(mall.annual_income, 1.5).value_counts()

In [ ]:
(get_upper_outliers(mall.annual_income, 1.5) == 4.25).sum()

In [ ]:
(get_upper_outliers(mall.annual_income, 1.5) == 4.25)

In [ ]:
def get_lower_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the lower outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the lower bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    return s.apply(lambda x: max([lower_bound - x, 0]))

In [ ]:
get_lower_outliers(mall.age, 1.5).sum()

In [ ]:
get_lower_outliers(mall.annual_income, 1.5).sum()

In [ ]:
get_lower_outliers(mall.spending_score, 1.5).sum()

4. Split data (train, validate, and test split).

In [ ]:
def mall_split(df):
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123)
    return train, validate, test

In [ ]:
train, validate, test = mall_split(mall)
train.shape, validate.shape, test.shape

5. Encode categorical columns using a one hot encoder.

In [ ]:
train.nunique()

In [ ]:
def one_hot_encoder(train, validate, test, cols):
    from sklearn.preprocessing import OneHotEncoder
    ohe = OneHotEncoder(sparse=False, categories='auto')

    # fit and transform train to create an array
    train_matrix = ohe.fit_transform(train[cols])

    # transform validate and test to create arrays
    validate_matrix = ohe.transform(validate[cols])
    test_matrix = ohe.transform(test[cols])
    
    # convert arrays to dataframes of encoded column
    train_ohe = pd.DataFrame(train_matrix, columns=ohe.categories_[0], index=train.index)
    validate_ohe = pd.DataFrame(validate_matrix, columns=ohe.categories_[0], index=validate.index)
    test_ohe = pd.DataFrame(test_matrix, columns=ohe.categories_[0], index=test.index)

    # join to the original datasets
    train = train.join(train_ohe)
    validate = validate.join(validate_ohe)
    test = test.join(test_ohe)
    return train, validate, test

In [ ]:
train, validate, test = one_hot_encoder(train, validate, test, ['gender'])
train.head()

6. Handles missing values.

In [ ]:
train.isna().sum()

In [ ]:
validate.isna().sum()

In [ ]:
test.isna().sum()

7. Scaling

In [ ]:
cols_to_scale = ['age', 'annual_income', 'spending_score', 'Male']

X_train = train[cols_to_scale]
X_validate = validate[cols_to_scale]
X_test = test[cols_to_scale]

In [ ]:
def min_max_scale(X_train, X_validate, X_test):
    # import scaler
    from sklearn.preprocessing import MinMaxScaler
    # Create scaler object
    scaler = MinMaxScaler(copy=True).fit(X_train)
    
    # tranform into scaled data (arrays)
    X_train_scaled = scaler.transform(X_train)
    X_validate_scaled = scaler.transform(X_validate)
    X_test_scaled = scaler.transform(X_test)
    
    # Create dataframes out of the scaled arrays that were generated by the scaler tranform.
    X_train_scaled = pd.DataFrame(X_train_scaled, 
                              columns=X_train.columns.values).\
                            set_index([X_train.index.values])

    X_validate_scaled = pd.DataFrame(X_validate_scaled, 
                                columns=X_validate.columns.values).\
                            set_index([X_validate.index.values])

    X_test_scaled = pd.DataFrame(X_test_scaled, 
                                columns=X_test.columns.values).\
                            set_index([X_test.index.values])
    return X_train_scaled, X_validate_scaled, X_test_scaled

In [ ]:
X_train_scaled, X_validate_scaled, X_test_scaled = min_max_scale(X_train, X_validate, X_test)
X_train_scaled.head()

### Test Final .py files

In [4]:
from wrangle_zillow import wrangle_zillow

wrangle_zillow().head()

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,parcelid,logerror,transactiondate
336,337,17051828,4.0,4.0,4.0,4969.0,4969.0,6111.0,4.0,34433200.0,...,779712.0,2228473.0,2016.0,1448761.0,24556.86,6.111001e+13,337,14688177,0.006783,2017-01-03
337,338,17052152,2.0,3.0,2.0,1800.0,1800.0,6111.0,2.0,34464778.0,...,319803.0,794940.0,2016.0,475137.0,8571.20,6.111001e+13,338,10930942,0.054386,2017-01-03
338,339,17052992,1.0,2.0,1.0,780.0,780.0,6111.0,1.0,34450686.0,...,70231.0,88731.0,2016.0,18500.0,1652.36,6.111001e+13,339,17246711,0.011061,2017-01-03
339,340,17053038,2.0,2.0,2.0,1250.0,1250.0,6111.0,2.0,34448639.0,...,79575.0,233424.0,2016.0,153849.0,3191.16,6.111001e+13,340,12559816,0.061977,2017-01-03
340,341,17053061,1.0,2.0,1.0,1404.0,1404.0,6111.0,1.0,34450089.0,...,99853.0,313844.0,2016.0,213991.0,3987.00,6.111001e+13,341,14673239,0.010089,2017-01-03


In [3]:
from wrangle_mall import wrangle_mall

train, validate, test = wrangle_mall()
train.head()

,age,annual_income,spending_score,Male
123,0.403846,0.442623,0.957447,1.0
76,0.519231,0.319672,0.553191,0.0
171,0.192308,0.590164,0.787234,1.0
10,0.942308,0.032787,0.138298,1.0
35,0.057692,0.147541,0.851064,0.0
